# Цели.

Цель данного ноутбука: сделать EDA и попробовать бейзлайн решение.

# EDA.

In [1]:
import pandas as pd

In [2]:
tab = pd.read_csv('file.csv', sep=';')

In [3]:
len(tab)

5449

In [4]:
tab.head()

,Unnamed: 0.1,Unnamed: 0,ucid,text_employer,date,ACTION_ITEM_RESULT_PRODUCT_NAME
0,138914,350836,"5,01039E+17",добрый_день. это ваш персональный_менеджер_сбе...,2021-02-26,Бизнес-карта
1,138915,350837,"5,01039E+17",добрый_день строки. это клиентский_менеджер_сб...,2021-02-20,Зарплатные проекты
2,138917,350844,"5,0104E+17",здравствуйте. это ваши персональный_менеджер и...,2021-02-18,Зарплатные проекты
3,138918,350851,"5,0104E+17",добрый_день. персональный_менеджер сбер бизнес...,2021-02-24,Зарплатные проекты
4,138921,350856,"5,01041E+17",в истории смотреть. добрый_день. менеджер_сбер...,2021-02-20,Зарплатные проекты


In [5]:
tab['ACTION_ITEM_RESULT_PRODUCT_NAME'].value_counts()

Бизнес-карта                  2508
Зарплатные проекты            1512
Открытие банковского счета     890
Эквайринг                      539
Name: ACTION_ITEM_RESULT_PRODUCT_NAME, dtype: int64

In [6]:
from processing import y_dict

In [7]:
y_dict

{'Зарплатные проекты': 2,
 'Бизнес-карта': 1,
 'Эквайринг': 4,
 'Открытие банковского счета': 3}

In [8]:
from processing import decode_to_target

In [9]:
tab['target'] = decode_to_target(tab)

In [10]:
tab['target'].value_counts()

1    2508
2    1512
3     890
4     539
Name: target, dtype: int64

In [11]:
tab['date'].value_counts()

2021-02-15    804
2021-02-16    743
2021-02-18    623
2021-02-24    620
2021-02-17    579
2021-02-19    576
2021-02-25    565
2021-02-20    457
2021-02-26    443
2021-02-23     18
2021-02-28     10
2021-02-21      6
2021-02-22      5
Name: date, dtype: int64

In [12]:
from processing import decode_to_true_date

Как видно, у нас есть данные за неделю. Со временем у нас всё меньше данных.

In [13]:
tab['true_date'] = decode_to_true_date(tab)

In [14]:
tab['true_date'].corr(tab['target'], method='spearman')

0.11487793402560045

Как видно корреляция со временем не очень большая. Поэтому потеря данных с течением времени не будет проблемой.

In [15]:
tab.iloc[:, 0].corr(tab['target'], method='spearman')

0.0017795657865167255

In [16]:
tab.iloc[:, 1].corr(tab['target'], method='spearman')

0.0017795657865167255

In [17]:
tab.iloc[:, 2].corr(tab['target'], method='spearman')

-0.05815313886455982

Здорово, что корреляция этих 3 столбцов не значима, потому что в дальнейшем мы их выкинем.

Займёмся обработкой текстового поля.

In [18]:
from lemmatization import prepare_lemmatized_text, remove_punctuation

In [19]:
tab['text_employer'] = remove_punctuation(tab)

In [20]:
tab['text'] = tab['text_employer'].apply(lambda x: prepare_lemmatized_text(x))

In [21]:
tab['text'][0]

'добрый_деть  это ваш персональный_менеджер_сбербанк бизнес звонить вы как индивидуальный_предприниматель вы удобный сейчас говорить  когда когда'

Отметим, что подчеркивание мы намеренно не убираем, так как оно показывает связь между словами.

Выделим трениворочное, валидационное и тестовое множества.

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
train_val, test = train_test_split(tab, random_state=239, test_size=0.2)
train, val = train_test_split(train_val, random_state=239, test_size=0.2)

Остаток EDA мы будем проводить на трейне.

Выделим стоп-слова

In [24]:
from processing import get_aggregated_statistics_on_words_for_category

In [25]:
aggtab1 = get_aggregated_statistics_on_words_for_category(train, 'Бизнес-карта')
aggtab2 = get_aggregated_statistics_on_words_for_category(train, 'Зарплатные проекты')
aggtab3 = get_aggregated_statistics_on_words_for_category(train, 'Эквайринг')
aggtab4 = get_aggregated_statistics_on_words_for_category(train, 'Открытие банковского счета')

In [26]:
words1 = aggtab1.head(30).index
words1 = [word[0] for word in words1]
print(words1)

['', 'карта', 'бизнес', 'звонить', 'счет', 'удобный', 'вопрос', 'наличные', 'сбер', 'свой', 'предложение', 'рубль', 'смс', 'расчетный', 'обслуживание', 'бесплатный', 'услуга', 'перезвонить', 'покупка', 'свидание', 'говорить', 'тысяча', 'добрый', 'э', 'снятие', 'отделение', 'заявка', 'оформить', 'уточнить', 'комиссия']


In [27]:
words2 = aggtab2.head(30).index
words2 = [word[0] for word in words2]
print(words2)

['', 'зарплатный_проект', 'вопрос', 'звонить', 'карта', 'бизнес', 'удобный', 'счет', 'услуга', 'сбер', 'комиссия', 'свой', 'понять', 'свидание', 'подключить', 'сбербанк', 'добрый', 'переводить', 'сотрудник', 'уточнить', 'говорить', 'тысяча', 'помочь', 'перезвонить', 'перевод', 'индивидуальный_предприниматель', 'информация', 'банк', 'э', 'код']


In [28]:
words3 = aggtab3.head(30).index
words3 = [word[0] for word in words3]
print(words3)

['', 'терминал', 'вопрос', 'бизнес', 'заявка', 'комиссия', 'оборот', 'звонить', 'сбер', 'удобный', 'месяц', 'услуга', 'понять', 'сбербанк', 'банк', 'тысяча', 'добрый', 'эквайринг', 'уточнить', 'торговый_эквайринг', 'э', 'свидание', 'счет', 'информация', 'точка', 'организация', 'нужный', 'оборудование', 'оплата', 'либо']


In [29]:
words4 = aggtab4.head(30).index
words4 = [word[0] for word in words4]
print(words4)

['', 'счет', 'вопрос', 'бизнес', 'звонить', 'банк', 'удобный', 'сбербанк', 'сбер', 'информация', 'понять', 'расчетный', 'организация', 'свой', 'услуга', 'добрый', 'уточнить', 'клиент', 'свидание', 'номер', 'контакт', 'направить', 'говорить', 'перезвонить', 'звать', 'смс', 'э', 'тариф', 'индивидуальный_предприниматель', 'добрый_деть']


# Построение бейзлайна.

В качестве бейзлайна можно взять поиск по популярному слову.

Пусть для "Зарплатные проекты" это будет зарплатный_проект.
Пусть для "Эквайринг" это будет эквайринг.
Пусть для "Открытие банковского счета" это будет счет.
Пусть для "Бизнес-карта" это будет карта.

Наиболее популярная категория у нас это Бизнес-карта, поэтому если ничего неизвестно мы будет пользоваться этой категорией.

Логика заполнения идёт от менее популярных категорий к более популярным.

In [30]:
def fill_baseline_predictions(tab, y_dict):
    tab['prediction'] = y_dict["Бизнес-карта"]
    tab['зарплатный_проект'] = tab['text'].apply(lambda x: "зарплатный_проект" in x.split(' '))
    tab['эквайринг'] = tab['text'].apply(lambda x: "эквайринг" in x.split(' '))
    tab['счет'] = tab['text'].apply(lambda x: "счет" in x.split(' '))
    tab['карта'] = tab['text'].apply(lambda x: "карта" in x.split(' '))
    tab.loc[tab['эквайринг'] == 1, 'prediction'] = y_dict['Эквайринг']
    tab.loc[tab['счет'] == 1, 'prediction'] = y_dict['Открытие банковского счета']
    tab.loc[tab['зарплатный_проект'] == 1, 'prediction'] = y_dict['Зарплатные проекты']
    tab.loc[tab['карта'] == 1, 'prediction'] = y_dict['Бизнес-карта']
    return tab

Рассмотрим тривиальный бейзлайн, когда мы всё предсказываем бизнес-картой.

В качестве метрики мы будем брать accuracy, так мы хотим использовать и нейронные сети, а в их имплементациях она есть из коробки.

In [31]:
print(y_dict["Бизнес-карта"])

1


In [32]:
len(train[train['target'] == 1])/len(train), len(val[val['target'] == 1])/len(val) 

(0.46458273587611126, 0.4426605504587156)

Итак тривиальный бейзлайн это 44.27%

In [33]:
train = fill_baseline_predictions(train, y_dict)

In [34]:
len(train[train["target"] == train["prediction"]])/len(train)

0.6303412675652423

In [35]:
val = fill_baseline_predictions(val, y_dict)

In [36]:
len(val[val["target"] == val["prediction"]])/len(val)

0.6135321100917431

Итак нетривиальный бейзлайн это 61.35%